# 02 - DC Bikeshare Data Cleaning & Feature Engineering

This notebook performs comprehensive data cleaning and creates derived features for analysis.

## Objectives
1. Load raw combined data
2. Standardize column names and data types
3. Remove invalid/outlier trips
4. Engineer temporal features (hour, day, season, etc.)
5. Create categorical features for analysis
6. Save cleaned dataset

---


## 1. Import Libraries


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")


Libraries imported successfully!


## 2. Load Raw Data


In [2]:
bikeshare_df = pd.read_parquet('../data/processed/bikeshare_raw_combined.parquet')

print(f"Loaded {len(bikeshare_df):,} records")
print(f"Columns: {list(bikeshare_df.columns)}")
original_count = len(bikeshare_df)


Loaded 705,343 records
Columns: ['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', 'source_file']


## 3. Convert Data Types


In [3]:
print("Converting timestamps...")
bikeshare_df['started_at'] = pd.to_datetime(bikeshare_df['started_at'])
bikeshare_df['ended_at'] = pd.to_datetime(bikeshare_df['ended_at'])

print("Calculating trip duration...")
bikeshare_df['duration_sec'] = (bikeshare_df['ended_at'] - bikeshare_df['started_at']).dt.total_seconds()
bikeshare_df['duration_min'] = bikeshare_df['duration_sec'] / 60

print(f"✓ Data types converted")
print(f"  Duration range: {bikeshare_df['duration_min'].min():.2f} to {bikeshare_df['duration_min'].max():.2f} minutes")


Converting timestamps...
Calculating trip duration...
✓ Data types converted
  Duration range: 0.00 to 1499.97 minutes


## 4. Data Cleaning - Remove Invalid Trips


In [4]:
print("Data Cleaning Criteria:")
print("=" * 60)

before_count = len(bikeshare_df)

too_short = (bikeshare_df['duration_sec'] < 60).sum()
print(f"1. Trips < 1 minute: {too_short:,}")

too_long = (bikeshare_df['duration_sec'] > 86400).sum()
print(f"2. Trips > 24 hours: {too_long:,}")

missing_start = bikeshare_df['start_station_name'].isna().sum()
print(f"3. Missing start station: {missing_start:,}")

missing_end = bikeshare_df['end_station_name'].isna().sum()
print(f"4. Missing end station: {missing_end:,}")

print("\n" + "=" * 60)
print("Applying filters...")

bikeshare_df = bikeshare_df[
    (bikeshare_df['duration_sec'] >= 60) &
    (bikeshare_df['duration_sec'] <= 86400) &
    (bikeshare_df['start_station_name'].notna()) &
    (bikeshare_df['end_station_name'].notna())
].copy()

after_count = len(bikeshare_df)
removed = before_count - after_count

print(f"\n✓ Cleaning complete:")
print(f"  Records removed: {removed:,} ({removed/before_count*100:.2f}%)")
print(f"  Records remaining: {after_count:,}")


Data Cleaning Criteria:
1. Trips < 1 minute: 18,295
2. Trips > 24 hours: 728
3. Missing start station: 202,208
4. Missing end station: 207,950

Applying filters...

✓ Cleaning complete:
  Records removed: 270,854 (38.40%)
  Records remaining: 434,489


## 5. Feature Engineering - Temporal Features


In [5]:
print("Creating temporal features...")

bikeshare_df['hour'] = bikeshare_df['started_at'].dt.hour
bikeshare_df['day_of_week'] = bikeshare_df['started_at'].dt.dayofweek
bikeshare_df['day_name'] = bikeshare_df['started_at'].dt.day_name()
bikeshare_df['month'] = bikeshare_df['started_at'].dt.month
bikeshare_df['month_name'] = bikeshare_df['started_at'].dt.month_name()
bikeshare_df['year'] = bikeshare_df['started_at'].dt.year
bikeshare_df['date'] = bikeshare_df['started_at'].dt.date

print("✓ Basic temporal features created:")
print(f"  Hour range: {bikeshare_df['hour'].min()} to {bikeshare_df['hour'].max()}")
print(f"  Days of week: {sorted(bikeshare_df['day_name'].unique())}")
print(f"  Months: {sorted(bikeshare_df['month_name'].unique())}")


Creating temporal features...
✓ Basic temporal features created:
  Hour range: 0 to 23
  Days of week: ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
  Months: ['July', 'June']


## 6. Feature Engineering - Season Classification


In [6]:
def get_season(month):
    """Classify month into meteorological season"""
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

bikeshare_df['season'] = bikeshare_df['month'].apply(get_season)

print("✓ Season classification created:")
print(bikeshare_df['season'].value_counts().sort_index())


✓ Season classification created:
season
Summer    434489
Name: count, dtype: int64


## 7. Feature Engineering - Rush Hour & Weekend Indicators


In [7]:
bikeshare_df['is_rush_hour'] = bikeshare_df['hour'].isin([7, 8, 9, 17, 18, 19])
bikeshare_df['is_weekend'] = bikeshare_df['day_of_week'].isin([5, 6])

rush_hour_trips = bikeshare_df['is_rush_hour'].sum()
weekend_trips = bikeshare_df['is_weekend'].sum()

print("✓ Categorical indicators created:")
print(f"  Rush hour trips: {rush_hour_trips:,} ({rush_hour_trips/len(bikeshare_df)*100:.1f}%)")
print(f"  Weekend trips: {weekend_trips:,} ({weekend_trips/len(bikeshare_df)*100:.1f}%)")


✓ Categorical indicators created:
  Rush hour trips: 181,168 (41.7%)
  Weekend trips: 109,142 (25.1%)


## 8. Feature Engineering - Time of Day Categories


In [8]:
def categorize_time(hour):
    """Categorize hour into time periods"""
    if 6 <= hour < 10:
        return 'Morning Rush'
    elif 10 <= hour < 16:
        return 'Midday'
    elif 16 <= hour < 20:
        return 'Evening Rush'
    elif 20 <= hour < 24:
        return 'Night'
    else:
        return 'Late Night/Early Morning'

bikeshare_df['time_category'] = bikeshare_df['hour'].apply(categorize_time)

print("✓ Time of day categories created:")
print(bikeshare_df['time_category'].value_counts())


✓ Time of day categories created:
time_category
Evening Rush                143250
Midday                      132105
Morning Rush                 79710
Night                        64987
Late Night/Early Morning     14437
Name: count, dtype: int64


## 9. Feature Engineering - Route Information


In [9]:
bikeshare_df['route'] = (
    bikeshare_df['start_station_name'] + 
    ' → ' + 
    bikeshare_df['end_station_name']
)

bikeshare_df['is_round_trip'] = (
    bikeshare_df['start_station_name'] == bikeshare_df['end_station_name']
)

round_trips = bikeshare_df['is_round_trip'].sum()
unique_routes = bikeshare_df['route'].nunique()

print("✓ Route features created:")
print(f"  Unique routes: {unique_routes:,}")
print(f"  Round trips: {round_trips:,} ({round_trips/len(bikeshare_df)*100:.2f}%)")


✓ Route features created:
  Unique routes: 76,420
  Round trips: 18,562 (4.27%)


## 10. Data Quality Summary


In [10]:
print("=" * 60)
print("DATA CLEANING & FEATURE ENGINEERING SUMMARY")
print("=" * 60)
print(f"\nOriginal records: {original_count:,}")
print(f"Final records: {len(bikeshare_df):,}")
print(f"Records retained: {len(bikeshare_df)/original_count*100:.2f}%")

print(f"\nFinal dataset columns: {len(bikeshare_df.columns)}")
print(f"Date range: {bikeshare_df['started_at'].min()} to {bikeshare_df['started_at'].max()}")

print("\nNew Features Created:")
new_features = [
    'duration_sec', 'duration_min', 'hour', 'day_of_week', 'day_name', 
    'month', 'month_name', 'year', 'date', 'season', 'is_rush_hour', 
    'is_weekend', 'time_category', 'route', 'is_round_trip'
]
for feat in new_features:
    print(f"  ✓ {feat}")

print("\nMissing values check:")
missing = bikeshare_df.isnull().sum()
if missing.sum() == 0:
    print("  ✓ No missing values!")
else:
    print(missing[missing > 0])


DATA CLEANING & FEATURE ENGINEERING SUMMARY

Original records: 705,343
Final records: 434,489
Records retained: 61.60%

Final dataset columns: 29
Date range: 2025-06-30 16:47:53.810000 to 2025-07-31 23:55:37.416000

New Features Created:
  ✓ duration_sec
  ✓ duration_min
  ✓ hour
  ✓ day_of_week
  ✓ day_name
  ✓ month
  ✓ month_name
  ✓ year
  ✓ date
  ✓ season
  ✓ is_rush_hour
  ✓ is_weekend
  ✓ time_category
  ✓ route
  ✓ is_round_trip

Missing values check:
end_station_id    59
dtype: int64


## 11. Preview Cleaned Data


In [11]:
print("Sample of cleaned data with new features:")
display_cols = [
    'started_at', 'start_station_name', 'end_station_name', 
    'duration_min', 'hour', 'day_name', 'season', 'time_category', 
    'is_rush_hour', 'is_weekend', 'member_casual'
]
bikeshare_df[display_cols].head(10)


Sample of cleaned data with new features:


,started_at,start_station_name,end_station_name,duration_min,hour,day_name,season,time_category,is_rush_hour,is_weekend,member_casual
0,2025-07-04 21:56:18.589,18th & C St NW,Wisconsin Ave & K St NW,15.546767,21,Friday,Summer,Night,False,False,member
1,2025-07-20 19:50:08.020,Potomac Ave & 8th St SE,M St & New Jersey Ave SE,3.590483,19,Sunday,Summer,Evening Rush,True,True,member
2,2025-07-15 00:08:36.191,9th & G St NW,6th St & Indiana Ave NW,4.007400,0,Tuesday,Summer,Late Night/Early Morning,False,False,member
3,2025-07-31 14:55:06.771,9th & G St NW,3rd & H St NW,9.312133,14,Thursday,Summer,Midday,False,False,member
4,2025-07-07 17:20:29.587,Tysons Metro South,Madron Ln & Bermudez Ct,15.135650,17,Monday,Summer,Evening Rush,True,False,member
5,2025-07-13 14:51:59.595,9th & N St NW,16th & Harvard St NW,11.685083,14,Sunday,Summer,Midday,False,True,member
6,2025-07-25 13:03:29.271,9th & G St NW,44th St & New Mexico Ave NW,43.017967,13,Friday,Summer,Midday,False,False,member
7,2025-07-12 00:03:19.465,9th & G St NW,19th St & Pennsylvania Ave NW,18.855867,0,Saturday,Summer,Late Night/Early Morning,False,True,casual
8,2025-07-24 17:09:03.853,18th St & Pennsylvania Ave NW,18th & New Hampshire Ave NW,7.678550,17,Thursday,Summer,Evening Rush,True,False,member
9,2025-07-30 16:23:21.310,18th St & Pennsylvania Ave NW,18th & New Hampshire Ave NW,7.666183,16,Wednesday,Summer,Evening Rush,False,False,member


## 12. Save Cleaned Data


In [12]:
output_path = '../data/processed/bikeshare_cleaned.parquet'
bikeshare_df.to_parquet(output_path, index=False)

print(f"✓ Cleaned data saved to: {output_path}")
print(f"  Records: {len(bikeshare_df):,}")
print(f"  Columns: {len(bikeshare_df.columns)}")
print(f"  File size: {os.path.getsize(output_path) / (1024**2):.2f} MB")
print("\nReady for statistical analysis in notebook 03!")


✓ Cleaned data saved to: ../data/processed/bikeshare_cleaned.parquet
  Records: 434,489
  Columns: 29


NameError: name 'os' is not defined